In [2]:
import pandas as pd
import nltk
import sqlite3

In [3]:
conn = sqlite3.connect('twitter.db')
df = pd.read_sql("SELECT * FROM covid19tweets", conn)
usa = pd.read_sql("SELECT * FROM covid19tweets WHERE instr(user_loc_display,'United States')>0",conn)
conn.close()

In [4]:
#clean up some of the dtypes
df = df.astype({'text': str, 'user_lat': float, 'user_lon': float})
df['created_at'] = pd.to_datetime(df['created_at'])

In [5]:
#dump all text into a long string
text_blob = ' '.join(df['text'].to_list())

In [10]:
from nltk import bigrams, trigrams, word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder

In [11]:
tokens = word_tokenize(text_blob)
text = nltk.Text(tokens)

In [12]:
text.concordance('Putin')

Displaying 5 of 5 matches:
nd selli… RT @ ClaireRChen : How did Putin deal with mask price surge ? It 's s
ancy Upd… RT @ ClaireRChen : How did Putin deal with mask price surge ? It 's s
rned pra… RT @ ClaireRChen : How did Putin deal with mask price surge ? It 's s
nds way too familiar ! Are Trump and Putin keeping crucial supplies for th… RT 
nds way too familiar ! Are Trump and Putin keeping crucial supplies for th… Can


In [13]:
bigram_measures = BigramAssocMeasures()
trigram_measures = TrigramAssocMeasures()

In [14]:
# took most of this section from nlp for hackers

finder = BigramCollocationFinder.from_words(tokens)
# only bigrams that appear 5+ times
finder.apply_freq_filter(5)
# return the 50 bigrams with the highest PMI (Pointwise Mutual Information)
#PMI is log(p(x, y)/(p(x)p(y)))
print("Bigrams")
print(finder.nbest(bigram_measures.pmi, 50))
# Compute length-3 collocations
finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
# only trigrams that appear 5+ times
finder.apply_freq_filter(5)
# return the 50 trigrams with the highest PMI
print("Trigrams")
print(finder.nbest(trigram_measures.pmi, 50))


Bigrams
[('Agha', 'Khan'), ('Gargling', 'salt'), ('salt', 'water'), ('Some', 'fast'), ('intensive', 'unit…'), ('write', 'inaccurately'), ('DAY', 'Worst'), ('ONE', 'DAY'), ('45-minute', 'turnaround'), ('8,000', 'extra'), ('CONTAIN', 'VIRUS'), ('CPAP', 'oxygen'), ('nearly', '120…'), ('oxygen', 'tent'), ('red', 'tap'), ('regulatory', 'red'), ('troops', 'supporting'), ('unfounded', 'rumors'), ('Washington', 'Post'), ('DEAD', 'IN'), ('earned', 'pra…'), ('fast', 'facts'), ('informal', 'settlements'), ('NationalGuard', 'troops'), ('USNavyCNO', 'discusses'), ('Very', 'powerful'), ('gather', 'physically'), ('assistance', 'of…'), ('stay', 'apart'), ('appeal', 'again'), ('hard-hit', 'states'), ('chart', 'shows'), ('faith', 'leaders'), ('IN', 'ONE'), ('EU', 'Member'), ('provide', '8,000'), ('BATTLE', 'TO'), ('IS', 'LOST'), ('TO', 'CONTAIN'), ('VIRUS', 'IS'), ('hear', 'unfounded'), ('maximize', 'efforts'), ('past', '24'), ('Video', 'Update'), ('beds', 'across'), ('England', 'agrees'), ('agrees', 'd

In [15]:
lower_tweets = df['text'].str.lower()
ngram_mask = lower_tweets.str.contains('not gather physically')

In [16]:
df[ngram_mask]

,retweet_count,created_at,text,coordinates,user_loc,user_lat,user_lon,user_loc_display,id,source
57,222,2020-03-21 20:02:06,"RT @JustinWelby: As UK faith leaders, we appea...",None,Launceston Cornwall,50.636506,-4.360443,"Launceston, Cornwall, South West England, Engl...",1241455062846787584,Twitter for iPhone
79,222,2020-03-21 20:02:03,"RT @JustinWelby: As UK faith leaders, we appea...",None,"Enugu, Nigeria",6.553609,7.414306,"Enugu, Nigeria",1241455049424896001,Twitter for Android
595,286,2020-03-21 20:40:05,"RT @JustinWelby: As UK faith leaders, we appea...",None,,NaN,NaN,None,1241464620537315330,Twitter for iPhone
887,328,2020-03-21 21:00:33,"RT @JustinWelby: As UK faith leaders, we appea...",None,,NaN,NaN,None,1241469771276746753,Twitter for Android
911,338,2020-03-21 21:07:27,"RT @JustinWelby: As UK faith leaders, we appea...",None,Witney,51.785937,-1.485059,"Witney, Oxfordshire, South East, England, OX28...",1241471507311853568,Twitter for iPhone
922,338,2020-03-21 21:07:25,"RT @JustinWelby: As UK faith leaders, we appea...",None,Aldgate High Street,51.514017,-0.075316,"Aldgate High Street, City of London, Greater L...",1241471497723621378,Twitter Web App
997,338,2020-03-21 21:07:11,"RT @JustinWelby: As UK faith leaders, we appea...",None,Little Kingshill,51.684956,-0.704058,"Little Kingshill, Buckinghamshire, South East,...",1241471440576303106,Twitter Web App


In [17]:
#drop duplicates to avoid RT messing up the ngram counts
df_no_RT = df.drop_duplicates('text')
#rerun the tokenization
text_blob_no_RT = ' '.join(df_no_RT['text'].to_list())
tokens_no_RT = word_tokenize(text_blob_no_RT)
text_no_RT = nltk.Text(tokens_no_RT)

In [18]:
finder = BigramCollocationFinder.from_words(tokens_no_RT)
# only bigrams that appear 5+ times
finder.apply_freq_filter(5)
# return the 50 bigrams with the highest PMI (Pointwise Mutual Information)
#PMI is log(p(x, y)/(p(x)p(y)))
print("Bigrams:")
print(finder.nbest(bigram_measures.pmi, 50))
# Compute length-3 collocations
finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens_no_RT)
# only trigrams that appear 5+ times
finder.apply_freq_filter(5)
# return the 50 trigrams with the highest PMI
print("Trigrams")
print(finder.nbest(trigram_measures.pmi, 50))

Bigrams:
[('South', 'Korea'), ('past', '24'), ('death', 'toll'), ('24', 'hours'), ('New', 'York'), ('social', 'distancing'), ('how', 'citizens'), ('health', 'care'), ('understand', 'how'), ('your', 'hands'), ('&', 'amp'), ('amp', ';'), ('deal', 'with'), ('Thank', 'you'), ('’', 'm'), ('’', 're'), ('’', 't'), ('’', 've'), ('’', 's'), ('confirmed', 'cases'), ('at', 'home'), ('don', '’'), ('has', 'been'), ('If', 'you'), ('has', 'confirmed'), ('a', 'lot'), ('It', "'s"), ('if', 'you'), ('Here', 'are'), ('will', 'be'), ('have', 'been'), ('new', 'cases'), ('test', 'that'), ('number', 'of'), ('``', 'We'), ('This', 'is'), ('there', 'are'), ('can', 'be'), ('part', 'of'), ('We', 'have'), ('during', 'this'), ('positive', 'for'), ('COVID19', 'case'), ('Due', 'to'), ('due', 'to'), ('to', 'combat'), ('and', 'other'), ('COVID19…', 'https'), ('that', 'can'), ('!', '!')]
Trigrams
[('past', '24', 'hours'), ('don', '’', 't'), ('&', 'amp', ';'), ('the', 'past', '24'), ('I', '’', 'm'), ('you', '’', 're'), ('

In [9]:
def bin_messages_containing(df, string, freq='5min'):
    """
    Selects the tweets that contain a given string, and then counts them in a given time step.
    """
    first_time = df['created_at'].min()
    last_time = df['created_at'].max()
    date_range = pd.date_range(first_time, last_time, freq = freq)
    
    lower_tweets = df['text'].str.lower()
    contains_mask = lower_tweets.str.contains(string)
    
    for index, _ in enumerate(date_range[:-1]):
        block_start = date_range[index]
        block_end = date_range[index + 1]
        filt = (df['created_at'] > block_start) & (df['created_at'] < block_end) & contains_mask
        print(str(len(df[filt])) + f' from {block_start} to {block_end}')

string = "putin"
bin_messages_containing(df, string, freq='10min')

1 from 2020-03-21 20:02:00 to 2020-03-21 20:12:00
0 from 2020-03-21 20:12:00 to 2020-03-21 20:22:00
2 from 2020-03-21 20:22:00 to 2020-03-21 20:32:00
2 from 2020-03-21 20:32:00 to 2020-03-21 20:42:00
0 from 2020-03-21 20:42:00 to 2020-03-21 20:52:00
0 from 2020-03-21 20:52:00 to 2020-03-21 21:02:00
0 from 2020-03-21 21:02:00 to 2020-03-21 21:12:00
0 from 2020-03-21 21:12:00 to 2020-03-21 21:22:00
